# Exercise: Geographical Cluster Analysis of Taxi Rides
Using the NY Taxi data set (see Use Case Block I) and the use case from the lecture...

In [1]:
import pandas as pd
import numpy as np
import folium


In [2]:
#check if notebook runs in colab
import sys
IN_COLAB = 'google.colab' in sys.modules
print('running in Colab:',IN_COLAB)
path='..'
if IN_COLAB:
  #in colab, we need to clone the data from the repo
  !git clone https://github.com/keuperj/DataScienceSS20.git
  path='DataScienceSS20'

running in Colab: False


In [3]:
# we load the data we have saved after wrangling and pre-processing in block I
train=pd.read_csv(path+'/DATA/train_cleaned.csv')

In [4]:
#select only the culumns with the ride coordinates
coordinates = train[ ['pickup_latitude','pickup_longitude','dropoff_latitude' , 'dropoff_longitude' ] ]

## Clustering approach from the lecture
we will be using simple K-Means:
https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html

In [5]:
from sklearn.cluster import KMeans

In [6]:
#define number of clusters and create instance
clusters=100
myKMeans=KMeans(n_clusters=clusters, n_jobs=-1)#parallelize to all cores

In [7]:
#train model
myKMeans.fit(coordinates.to_numpy()[:100000,:])#use only subset of the data to make it faster

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=100, n_init=10, n_jobs=-1, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [8]:
#get cluster centers
centers=myKMeans.cluster_centers_
    

In [9]:
#draw map: green: start, red: end
cluster_map = folium.Map(location = [40.730610,-73.935242],zoom_start = 12,)
for i in range(clusters):
    folium.CircleMarker([centers[i,0], centers[i,1]], radius=3,                
                        color="green", 
                        fill_opacity=0.9
                       ).add_to(cluster_map)
    folium.CircleMarker([centers[i,2], centers[i,3]], radius=3,                
                        color="red", 
                        fill_opacity=0.9
                       ).add_to(cluster_map)
    folium.PolyLine([ [centers[i,0],centers[i,1]] , [centers[i,2],centers[i,3]]  ], color="black", weight=2.5, opacity=1).add_to(cluster_map)

In [10]:
cluster_map

## Exercise 1
Write a function ```show_cluster(cluster_number,...)``` that draws the cluster centers and all start and end points of a given cluster in the map.
* use the ```predict()``` method to map all data in ```train_data``` to a cluster center
* use ```folium.CircleMarker``` to draw all members of a given cluster


In [11]:
# first, use the model to predict the cluster assignment of all data points
predictions = myKMeans.predict(coordinates.to_numpy())

In [12]:
#result gives a cluster number per coordiante
# look at the first 10 entries
predictions[:10]

array([91, 56, 50, 72, 15, 83, 57, 61, 89, 22], dtype=int32)

In [24]:

def show_cluster(cluster_number, predictions, centers):
    #get coordinates of the target cluster
    #coordinates were all coordinate collumns from the train data
    cluster_coord = coordinates.to_numpy()[predictions==cluster_number]
    
    #get number of entries:
    entries = np.shape(cluster_coord)[0]
    print("number of cluster enries:" , entries)
    
    #create map
    cluster_map = folium.Map(location = [40.730610,-73.935242],zoom_start = 12,)
    
    #plot centers
    folium.PolyLine([ [centers[cluster_number,0],centers[cluster_number,1]] , [centers[cluster_number,2],centers[cluster_number,3]]  ], color="black", weight=2.5, opacity=1).add_to(cluster_map)
    
    for i in range(entries):
        folium.CircleMarker([cluster_coord[i,0], cluster_coord[i,1]], radius=3,                
                        color="green", 
                        fill_opacity=0.9
                       ).add_to(cluster_map)
        folium.CircleMarker([cluster_coord[i,2], cluster_coord[i,3]], radius=3,                
                        color="red", 
                        fill_opacity=0.9
                       ).add_to(cluster_map)
    return cluster_map

In [43]:
show_cluster(76,predictions,centers)

number of cluster enries: 1


## Exercise 2
Write a function ```cluster_var(cluster_number,...)``` that computes the intra- and extra cluster variance for a given cluster. Apply it to all clusters and compare the results for k=100 and k=10.

In [44]:
def show_var(predictions, k):
    #get coordinates of the target cluster
    #coordinates were all coordinate collumns from the train data
    np_coord = coordinates.to_numpy()
    
    #compute min and max coordiantes for normalization
    min_cor = np.min(np_coord, axis=0)
    max_cor = np.max(np_coord, axis=0)
    dist = max_cor - min_cor
    print ("max distances:", dist)
    for i in range(k):
        cluster_coord = np_coord[predictions==i]
    
        #compute variances
        var_all = np.var(cluster_coord, axis=0)
        print(i, var_all)

In [45]:
show_var(predictions,100)

max distances: [1.073698 1.177225 0.970017 1.169471]
0 [4.90641980e-05 4.32481076e-05 4.03765374e-05 2.83654386e-05]
1 [3.13244320e-05 2.34185592e-05 2.90150547e-05 2.90100605e-05]
2 [6.51968182e-05 1.26212525e-04 1.37501943e-04 1.71725804e-04]
3 [9.79454088e-05 2.88233672e-03 7.75491726e-05 1.09113459e-03]
4 [0.0001508  0.00016978 0.00016438 0.0002876 ]
5 [5.94323814e-05 8.46719463e-05 5.42162954e-05 8.40858293e-05]
6 [6.37108715e-05 4.95106225e-05 6.56896097e-05 5.24397250e-05]
7 [3.51498784e-05 2.93348462e-05 2.50684489e-05 2.73047289e-05]
8 [0.00056468 0.00021126 0.00045744 0.00065694]
9 [0.00293911 0.00454012 0.00098063 0.00242336]
10 [2.32931886e-04 1.81788446e-04 5.53789499e-05 9.66520081e-05]
11 [3.67667894e-05 4.36145504e-05 3.34197553e-05 4.05969129e-05]
12 [4.12483774e-05 5.13525826e-05 3.76808369e-05 3.46967232e-05]
13 [0.00027064 0.00014096 0.00012772 0.00013171]
14 [2.77157224e-04 9.61480191e-05 2.14960011e-04 2.74870166e-04]
15 [2.23615007e-05 2.37867599e-05 3.75852688e-